In [7]:
import pandas as pd

In [1]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [2]:
from neo4j import GraphDatabase as gd

In [5]:
class abc:
    def __init__(self,uri,user,password):
        self.driver=gd.driver(uri,auth=(user,password))
    
    def close(self):
        self.driver.close()
        
    def print_greeting(self,message):
        with self.driver.session() as session:
            greeting = session.execute_write(self._create_and_return_greeting, message)
            print(greeting)
    
    @staticmethod
    def _create_and_return_greeting(tx,message):
        result =tx.run("CREATE (a:Greeting) " 
                      "SET a.message =$message "
                       "RETURN a.message + ', from node ' + id(a)",message=message)
        
        return result.single()[0]


if __name__ =='__main__':
    greeter=abc("bolt://localhost:7687","neo4j","admin123")
    greeter.print_greeting("Hello World")
    greeter.close()

Hello World, from node 0


In [6]:
#create neo4j graph using from a csv file

In [10]:
driver=gd.driver(uri="bolt://localhost:7687",auth=("neo4j","admin123"))

In [11]:
driver

In [29]:
session=driver.session()

In [14]:
df=pd.read_csv('stations.csv')

In [15]:
df

,URL,OBJECTID,NAME,the_geom,LINE,NOTES
0,http://web.mta.info/nyct/service/,1,Astor Pl,POINT (-73.99106999861966 40.73005400028978),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,http://web.mta.info/nyct/service/,2,Canal St,POINT (-74.00019299927328 40.71880300107709),4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,http://web.mta.info/nyct/service/,3,50th St,POINT (-73.98384899986625 40.76172799961419),1-2,"1-all times, 2-nights"
3,http://web.mta.info/nyct/service/,4,Bergen St,POINT (-73.97499915116808 40.68086213682956),2-3-4,"4-nights, 3-all other times, 2-all times"
4,http://web.mta.info/nyct/service/,5,Pennsylvania Ave,POINT (-73.89488591154061 40.66471445143568),3-4,"4-nights, 3-all other times"
...,...,...,...,...,...,...
468,http://web.mta.info/nyct/service/,469,Coney Island - Stillwell Av,POINT (-73.9812359981396 40.57728100006751),D-F-N-Q,"D,F,N,Q-all times"
469,http://web.mta.info/nyct/service/,470,34th St - Hudson Yards,POINT (-74.00219709442206 40.75544635961596),7-7 Express,"7-all times, 7 Express-rush hours AM westbound..."
470,http://web.mta.info/nyct/service/,641,72nd St,POINT (-73.95836178682246 40.76880251014895),Q,Q-all times
471,http://web.mta.info/nyct/service/,642,86th St,POINT (-73.95177090964917 40.77786104333163),Q,Q-all times


In [19]:
#get distinct trains
trains=df['LINE'].unique()

In [20]:
(trains)

array(['4-6-6 Express', '1-2', '2-3-4', '3-4', '1', 'A-B-C', 'A-C',
       'E-M-R', 'J-M-Z', 'J-Z', 'B-D-N-Q-R', 'A-S', '2-5', 'B-D',
       '7-7 Express', '6-6 Express', '4-5-6-6 Express', 'D-N-R',
       '2-3-4-5', '4', '6', 'A', 'B-D-F-M', 'B-Q-S', 'D', 'E', 'E-F',
       'E-J-Z', 'E-M', 'F', 'G', 'L', 'J', 'J-M', 'N', 'N-W', 'N-Q-R-W',
       'S', 'Q', '7-7 Express-N-W', 'F-G', 'B-Q', 'D-N-Q-R', '4-5', '5',
       '1-2-3', '2-3', '3', '7', 'F-Q', 'A-C-E', 'M', 'E-F-M-R', '2',
       'A-B-C-D', 'R', 'B-D-E', 'N-R-W', 'F-M', 'A-C-G', 'A-C-F', 'N-R',
       'R-W', 'N-Q', 'A-C-S', 'D-F-N-Q'], dtype=object)

###  GRAPH IS CREATED

In [37]:
# don't want to duplicate LINE and NAME so need to make a constraint
#run query q1
q1='''
MERGE(A:TRAIN{train_name:$train_name})
MERGE(B:STATION{station_name:$station_name})
MERGE (A)-[:GOES_TO]->(B)
'''

for tnames in trains:
    #groupby name and line anf get names of train names
    x=df.groupby("LINE")[['NAME','LINE']].get_group(tnames)["NAME"]
    #iterating through x to find the stations
    for sname in x:
        p={'train_name':tnames,"station_name":sname}
        session.run(q1,p)

###  Data analysis using neo4j graph

In [58]:
sns=df["NAME"].unique()

In [59]:
len(sns)

355

In [60]:
q2="""
MATCH (train)-[:GOES_TO]->(station)
return train,station
"""

In [61]:
results=session.run(q2)

In [62]:
data=results.data()

In [63]:
data

[{'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': 'Astor Pl'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': 'Canal St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '33rd St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '96th St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '77th St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '51st St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '23rd St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '68th St - Hunter College'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '28th St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': '110th St'}},
 {'train': {'train_name': '4-6-6 Express'},
  'station': {'station_name': 'Bleecker St'}},
 {'train': {'train_

In [64]:
df1=pd.DataFrame(data)

In [65]:
df1

,train,station
0,{'train_name': '4-6-6 Express'},{'station_name': 'Astor Pl'}
1,{'train_name': '4-6-6 Express'},{'station_name': 'Canal St'}
2,{'train_name': '4-6-6 Express'},{'station_name': '33rd St'}
3,{'train_name': '4-6-6 Express'},{'station_name': '96th St'}
4,{'train_name': '4-6-6 Express'},{'station_name': '77th St'}
...,...,...
468,{'train_name': 'R-W'},{'station_name': 'Rector St'}
469,{'train_name': 'R-W'},{'station_name': 'Cortlandt St'}
470,{'train_name': 'N-Q'},{'station_name': 'Canal St'}
471,{'train_name': 'A-C-S'},{'station_name': 'Euclid Ave'}


In [66]:
df1["train"][0]["train_name"]

'4-6-6 Express'

In [67]:
for i in range(len(df1['train'])):
    df1['train'][i]=df1["train"][i]["train_name"]

In [68]:
df1

,train,station
0,4-6-6 Express,{'station_name': 'Astor Pl'}
1,4-6-6 Express,{'station_name': 'Canal St'}
2,4-6-6 Express,{'station_name': '33rd St'}
3,4-6-6 Express,{'station_name': '96th St'}
4,4-6-6 Express,{'station_name': '77th St'}
...,...,...
468,R-W,{'station_name': 'Rector St'}
469,R-W,{'station_name': 'Cortlandt St'}
470,N-Q,{'station_name': 'Canal St'}
471,A-C-S,{'station_name': 'Euclid Ave'}


In [69]:
#cleaning data
for i in range(len(df1['station'])):
    df1['station'][i]=df1["station"][i]['station_name']

In [70]:
df1

,train,station
0,4-6-6 Express,Astor Pl
1,4-6-6 Express,Canal St
2,4-6-6 Express,33rd St
3,4-6-6 Express,96th St
4,4-6-6 Express,77th St
...,...,...
468,R-W,Rector St
469,R-W,Cortlandt St
470,N-Q,Canal St
471,A-C-S,Euclid Ave


In [72]:
# train names as per the connectivity count
train__names=df1['train'].value_counts().sort_values(ascending=False)

In [73]:
train__names

2-5                26
L                  24
F                  21
1                  20
A-C                18
                   ..
7-7 Express-N-W     1
N-Q                 1
2                   1
B-D-E               1
B-Q-S               1
Name: train, Length: 66, dtype: int64

In [74]:
# station name as per the connectivity counts
station__names=df1['station'].value_counts().sort_values(ascending=False)

In [75]:
station__names

Canal St                             5
23rd St                              5
Fulton St                            5
86th St                              5
96th St                              4
                                    ..
Ave M                                1
34th St - Hudson Yards               1
225th St                             1
Brook Ave                            1
Middle Village - Metropolitan Ave    1
Name: station, Length: 355, dtype: int64

In [93]:
# query to count the total
q3='''
match(a:TRAIN)-[:GOES_TO]->(b:STATION)
with a AS Train_Name,collect(b) as Station,count(*) as count
return Train_Name,Station,count
'''


In [94]:
results_a=session.run(q3)

In [95]:
data_new=results_a.data()

In [96]:
df2=pd.DataFrame(data_new)

In [104]:
df2

,Train_Name,Station,count
0,{'train_name': '4-6-6 Express'},"[{'station_name': 'Spring St'}, {'station_name...",14
1,{'train_name': '1-2'},"[{'station_name': 'Franklin St'}, {'station_na...",12
2,{'train_name': '2-3-4'},[{'station_name': 'Eastern Pkwy - Bklyn Museum...,3
3,{'train_name': '3-4'},"[{'station_name': 'Nostrand Ave'}, {'station_n...",10
4,{'train_name': '1'},"[{'station_name': 'Cortlandt St'}, {'station_n...",20
...,...,...,...
61,{'train_name': 'N-R'},"[{'station_name': '53rd St'}, {'station_name':...",5
62,{'train_name': 'R-W'},"[{'station_name': 'Cortlandt St'}, {'station_n...",5
63,{'train_name': 'N-Q'},[{'station_name': 'Canal St'}],1
64,{'train_name': 'A-C-S'},[{'station_name': 'Euclid Ave'}],1


In [106]:
for i in range(len(df2['Train_Name'])):
    df2['Train_Name'][i]=df2["Train_Name"][i]["train_name"]

<ipython-input-106-76b608f557df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Train_Name'][i]=df2["Train_Name"][i]["train_name"]


In [111]:
df2

,Train_Name,Station,count
0,4-6-6 Express,"[{'station_name': 'Spring St'}, {'station_name...",14
1,1-2,"[{'station_name': 'Franklin St'}, {'station_na...",12
2,2-3-4,[{'station_name': 'Eastern Pkwy - Bklyn Museum...,3
3,3-4,"[{'station_name': 'Nostrand Ave'}, {'station_n...",10
4,1,"[{'station_name': 'Cortlandt St'}, {'station_n...",20
...,...,...,...
61,N-R,"[{'station_name': '53rd St'}, {'station_name':...",5
62,R-W,"[{'station_name': 'Cortlandt St'}, {'station_n...",5
63,N-Q,[{'station_name': 'Canal St'}],1
64,A-C-S,[{'station_name': 'Euclid Ave'}],1


In [112]:
len(df2["Train_Name"].unique())

66

In [85]:
df2

,Train_Name,Station,count
0,None,"[{'station_name': 'Astor Pl'}, {'station_name'...",473


In [114]:
df2.to_csv("train_vs_station.csv",index=False)